# Valuation Module

This notebook demonstrates the valuation utilities from `specparser.amt.valuation`.

The valuation module provides functions for:
- Computing actions (entry/expiry triggers) for straddles
- Pricing models (ES, NS, BS)
- PnL calculations (mark-to-market, delta, hedge PnL)

## Pipeline Overview

```
tickers.py          prices.py              valuation.py
     │                   │                      │
filter_tickers() ──────► get_prices() ────────► actions()
                              │                      │
                              │                      ▼
                              │              get_straddle_actions()
                              │                      │
                              │                      ▼
                              │              get_straddle_valuation()
```

## Table of Contents

1. [Setup](#1.-Setup) - Imports and configuration
2. [Actions](#2.-Actions) - `actions`, `get_straddle_actions`
3. [Valuation](#3.-Valuation) - `get_straddle_valuation`
4. [Pricing Models](#4.-Pricing-Models) - `model_ES`, `model_NS`, `model_BS`
5. [Override Expiry](#5.-Override-Expiry) - Expiry date overrides
6. [Cache Management](#6.-Cache-Management) - `clear_override_cache`
7. [Benchmark](#7.-Benchmark) - Performance benchmarks
8. [Summary](#8.-Summary) - Function reference

In [1]:
# Setup: Imports
import timeit
from specparser.amt import (
    # Price loading (from prices.py)
    load_all_prices,
    set_prices_dict,
    get_prices,
    clear_prices_dict,
    # Valuation functions (from valuation.py)
    actions,
    get_straddle_actions,
    get_straddle_valuation,
    clear_override_cache,
    # Pricing models (from valuation.py)
    model_ES,
    model_NS,
    model_BS,
    MODEL_DISPATCH,
    # Table utilities (from table.py)
    show_table,
    table_to_rows,
    table_column,
)

# Data paths (relative to this notebook)
AMT_PATH = "../data/amt.yml"
PRICES_PATH = "../data/prices.parquet"
CHAIN_PATH = None  # "../data/futs.csv"
OVERRIDE_PATH = "../data/overrides.csv"

# Load prices for all examples
prices_dict = load_all_prices(PRICES_PATH)
set_prices_dict(prices_dict)
print(f"Loaded {len(prices_dict):,} price records")

Loaded 8,489,913 price records


---
## 2. Actions

Actions determine when to enter and exit straddle positions.

### `actions(prices_table, path, overrides_csv)`

Add action, model, and strike columns to a prices table.

**Output columns added:**
- `action`: Entry/expiry trigger (`ntry`, `xpry`, or `-`)
- `model`: Pricing model code (ES, NS, BS)
- `strike_vol`: Volatility at strike setting
- `strike`: Strike price
- `expiry`: Expiry date

In [2]:
# First get prices for a straddle
prices_table = get_prices(
    "CL Comdty",
    2024,
    6,
    0,  # First straddle component
    AMT_PATH,
    CHAIN_PATH,
    PRICES_PATH
)

print(f"Prices table: {len(prices_table['rows'])} rows")
print(f"Columns: {prices_table['columns']}")

Prices table: 61 rows
Columns: ['asset', 'straddle', 'date', 'vol', 'hedge']


In [3]:
# Add action columns
actions_table = actions(prices_table, AMT_PATH, OVERRIDE_PATH)

print(f"Actions table columns: {actions_table['columns']}")
show_table(actions_table)

Actions table columns: ['asset', 'straddle', 'date', 'vol', 'hedge', 'action', 'model', 'strike_vol', 'strike', 'expiry']


,asset,straddle,date,vol,hedge,action,model,strike_vol,strike,expiry
0,CL Comdty,|2024-05|2024-06|N|0|OVERRIDE||33.3|,2024-05-01,26.36,78.44,-,ES,-,-,-
1,CL Comdty,|2024-05|2024-06|N|0|OVERRIDE||33.3|,2024-05-02,25.11,78.48,-,ES,-,-,-
2,CL Comdty,|2024-05|2024-06|N|0|OVERRIDE||33.3|,2024-05-03,24.85,77.76,-,ES,-,-,-
3,CL Comdty,|2024-05|2024-06|N|0|OVERRIDE||33.3|,2024-05-04,none,none,-,ES,-,-,-
4,CL Comdty,|2024-05|2024-06|N|0|OVERRIDE||33.3|,2024-05-05,none,none,-,ES,-,-,-
...,...,...,...,...,...,...,...,...,...,...
56,CL Comdty,|2024-05|2024-06|N|0|OVERRIDE||33.3|,2024-06-26,23.61,none,-,ES,-,-,-
57,CL Comdty,|2024-05|2024-06|N|0|OVERRIDE||33.3|,2024-06-27,23.97,none,-,ES,-,-,-
58,CL Comdty,|2024-05|2024-06|N|0|OVERRIDE||33.3|,2024-06-28,24.45,none,-,ES,-,-,-
59,CL Comdty,|2024-05|2024-06|N|0|OVERRIDE||33.3|,2024-06-29,none,none,-,ES,-,-,-


### `get_straddle_actions(underlying, year, month, i, path, chain_csv, prices_parquet, overrides_csv)`

Convenience function that combines `get_prices()` + `actions()` in one call.

In [4]:
# Get straddle actions in one call
full_table = get_straddle_actions(
    "CL Comdty",
    2024,
    6,
    0,
    AMT_PATH,
    CHAIN_PATH,
    PRICES_PATH,
    OVERRIDE_PATH
)

print(f"Full straddle actions: {len(full_table['rows'])} rows")
show_table(full_table)

Full straddle actions: 61 rows


,asset,straddle,date,vol,hedge,action,model,strike_vol,strike,expiry
0,CL Comdty,|2024-05|2024-06|N|0|OVERRIDE||33.3|,2024-05-01,26.36,78.44,-,ES,-,-,-
1,CL Comdty,|2024-05|2024-06|N|0|OVERRIDE||33.3|,2024-05-02,25.11,78.48,-,ES,-,-,-
2,CL Comdty,|2024-05|2024-06|N|0|OVERRIDE||33.3|,2024-05-03,24.85,77.76,-,ES,-,-,-
3,CL Comdty,|2024-05|2024-06|N|0|OVERRIDE||33.3|,2024-05-04,none,none,-,ES,-,-,-
4,CL Comdty,|2024-05|2024-06|N|0|OVERRIDE||33.3|,2024-05-05,none,none,-,ES,-,-,-
...,...,...,...,...,...,...,...,...,...,...
56,CL Comdty,|2024-05|2024-06|N|0|OVERRIDE||33.3|,2024-06-26,23.61,none,-,ES,-,-,-
57,CL Comdty,|2024-05|2024-06|N|0|OVERRIDE||33.3|,2024-06-27,23.97,none,-,ES,-,-,-
58,CL Comdty,|2024-05|2024-06|N|0|OVERRIDE||33.3|,2024-06-28,24.45,none,-,ES,-,-,-
59,CL Comdty,|2024-05|2024-06|N|0|OVERRIDE||33.3|,2024-06-29,none,none,-,ES,-,-,-


In [5]:
# Find entry and expiry days
action_col = table_column(full_table, "action")

print("Action days:")
for i, row in enumerate(full_table['rows']):
    action = row[full_table['columns'].index('action')]
    if action != '-':
        date = row[full_table['columns'].index('date')]
        print(f"  {date}: {action}")

Action days:
  2024-05-16: ntry
  2024-06-14: xpry


---
## 3. Valuation

### `get_straddle_valuation(underlying, year, month, i, path, chain_csv, prices_parquet, overrides_csv)`

Get daily valuation (PnL) for a straddle.

**Additional columns:**
- `mv`: Mark-to-market value
- `delta`: Option delta
- `opnl`: Option PnL (change in mv)
- `hpnl`: Hedge PnL
- `pnl`: Total PnL (opnl + hpnl)

In [6]:
# Get straddle valuation
valuation_table = get_straddle_valuation(
    "CL Comdty",
    2024,
    6,
    0,
    AMT_PATH,
    CHAIN_PATH,
    PRICES_PATH,
    OVERRIDE_PATH
)

print(f"Valuation columns: {valuation_table['columns']}")
show_table(valuation_table)

Valuation columns: ['asset', 'straddle', 'date', 'vol', 'hedge', 'action', 'model', 'strike_vol', 'strike', 'expiry', 'mv', 'delta', 'opnl', 'hpnl', 'pnl']


,asset,straddle,date,vol,hedge,action,model,strike_vol,strike,expiry,mv,delta,opnl,hpnl,pnl
0,CL Comdty,|2024-05|2024-06|N|0|OVERRIDE||33.3|,2024-05-01,26.36,78.44,-,ES,-,-,-,-,-,-,-,-
1,CL Comdty,|2024-05|2024-06|N|0|OVERRIDE||33.3|,2024-05-02,25.11,78.48,-,ES,-,-,-,-,-,-,-,-
2,CL Comdty,|2024-05|2024-06|N|0|OVERRIDE||33.3|,2024-05-03,24.85,77.76,-,ES,-,-,-,-,-,-,-,-
3,CL Comdty,|2024-05|2024-06|N|0|OVERRIDE||33.3|,2024-05-04,none,none,-,ES,-,-,-,-,-,-,-,-
4,CL Comdty,|2024-05|2024-06|N|0|OVERRIDE||33.3|,2024-05-05,none,none,-,ES,-,-,-,-,-,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,CL Comdty,|2024-05|2024-06|N|0|OVERRIDE||33.3|,2024-06-26,23.61,none,-,ES,-,-,-,-,-,-,-,-
57,CL Comdty,|2024-05|2024-06|N|0|OVERRIDE||33.3|,2024-06-27,23.97,none,-,ES,-,-,-,-,-,-,-,-
58,CL Comdty,|2024-05|2024-06|N|0|OVERRIDE||33.3|,2024-06-28,24.45,none,-,ES,-,-,-,-,-,-,-,-
59,CL Comdty,|2024-05|2024-06|N|0|OVERRIDE||33.3|,2024-06-29,none,none,-,ES,-,-,-,-,-,-,-,-


In [7]:
# Valuation for a currency
fx_valuation = get_straddle_valuation(
    "EURUSD Curncy",
    2024,
    1,
    0,
    AMT_PATH,
    CHAIN_PATH,
    PRICES_PATH,
    OVERRIDE_PATH
)

print(f"EURUSD valuation: {len(fx_valuation['rows'])} rows")
show_table(fx_valuation)

EURUSD valuation: 62 rows


,asset,straddle,date,vol,hedge,action,model,strike_vol,strike,expiry,mv,delta,opnl,hpnl,pnl
0,EURUSD Curncy,|2023-12|2024-01|N|0|BD|1|25|,2023-12-01,6.45,1.0884,ntry,ES,6.45,1.0884,2024-01-01,0.014997808258843613,0.0074989041294217795,0,0,0
1,EURUSD Curncy,|2023-12|2024-01|N|0|BD|1|25|,2023-12-02,none,none,-,ES,6.45,1.0884,2024-01-01,0.014753932211197349,0.007376966105598681,-0.00024387604764626418,-0.0,-0.00024387604764626418
2,EURUSD Curncy,|2023-12|2024-01|N|0|BD|1|25|,2023-12-03,none,none,-,ES,6.45,1.0884,2024-01-01,0.01450595619160557,0.007252978095802831,-0.0002479760195917783,-0.0,-0.0002479760195917783
3,EURUSD Curncy,|2023-12|2024-01|N|0|BD|1|25|,2023-12-04,6.8,1.0836,-,ES,6.45,1.0884,2024-01-01,0.015404949007639702,-0.17822304270199096,0.0008989928160341321,3.19866729693629e-05,0.000930979489003495
4,EURUSD Curncy,|2023-12|2024-01|N|0|BD|1|25|,2023-12-05,6.9,1.0797,-,ES,6.45,1.0884,2024-01-01,0.016256740212514612,-0.3242473452339548,0.0008517912048749098,-0.0006386161949078718,0.00021317500996703796
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,EURUSD Curncy,|2023-12|2024-01|N|0|BD|1|25|,2024-01-27,none,none,-,ES,-,-,-,-,-,-,-,-
58,EURUSD Curncy,|2023-12|2024-01|N|0|BD|1|25|,2024-01-28,none,none,-,ES,-,-,-,-,-,-,-,-
59,EURUSD Curncy,|2023-12|2024-01|N|0|BD|1|25|,2024-01-29,6.45,1.0833,-,ES,-,-,-,-,-,-,-,-
60,EURUSD Curncy,|2023-12|2024-01|N|0|BD|1|25|,2024-01-30,6.475,1.0845,-,ES,-,-,-,-,-,-,-,-


In [8]:
# Valuation for a metal commodity
la_valuation = get_straddle_valuation(
    "LA Comdty",
    2024,
    1,
    0,
    AMT_PATH,
    CHAIN_PATH,
    PRICES_PATH,
    OVERRIDE_PATH
)

print(f"LA Comdty valuation: {len(la_valuation['rows'])} rows")
show_table(la_valuation)

LA Comdty valuation: 62 rows


,asset,straddle,date,vol,hedge,action,model,strike_vol,strike,expiry,mv,delta,opnl,hpnl,pnl
0,LA Comdty,|2023-12|2024-01|N|0|OVERRIDE||33.3|,2023-12-01,13.38,2196.5,-,ES,-,-,-,-,-,-,-,-
1,LA Comdty,|2023-12|2024-01|N|0|OVERRIDE||33.3|,2023-12-02,none,none,-,ES,-,-,-,-,-,-,-,-
2,LA Comdty,|2023-12|2024-01|N|0|OVERRIDE||33.3|,2023-12-03,none,none,-,ES,-,-,-,-,-,-,-,-
3,LA Comdty,|2023-12|2024-01|N|0|OVERRIDE||33.3|,2023-12-04,15.36,2166.25,-,ES,-,-,-,-,-,-,-,-
4,LA Comdty,|2023-12|2024-01|N|0|OVERRIDE||33.3|,2023-12-05,15.22,2145.5,-,ES,-,-,-,-,-,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,LA Comdty,|2023-12|2024-01|N|0|OVERRIDE||33.3|,2024-01-27,none,none,-,ES,-,-,-,-,-,-,-,-
58,LA Comdty,|2023-12|2024-01|N|0|OVERRIDE||33.3|,2024-01-28,none,none,-,ES,-,-,-,-,-,-,-,-
59,LA Comdty,|2023-12|2024-01|N|0|OVERRIDE||33.3|,2024-01-29,18.68,none,-,ES,-,-,-,-,-,-,-,-
60,LA Comdty,|2023-12|2024-01|N|0|OVERRIDE||33.3|,2024-01-30,none,none,-,ES,-,-,-,-,-,-,-,-


---
## 4. Pricing Models

The valuation module supports multiple pricing models.

### `MODEL_DISPATCH`

Dict mapping model names to functions.

In [9]:
# Available models
print("Available pricing models:")
for name, func in MODEL_DISPATCH.items():
    print(f"  {name}: {func.__name__}")

Available pricing models:
  ES: model_ES
  NS: model_NS
  BS: model_BS
  CDS_ES: model_ES


### Model Descriptions

| Model | Name | Description |
|-------|------|-------------|
| `CDS_ES` | Expected Shortfall | For CDS-based assets |
| `NS` | Normal Spread | Normal distribution spread model |
| `BS` | Black-Scholes | Standard Black-Scholes model |

Each model takes a row dict with pricing parameters and returns:
- `mv`: Mark-to-market value
- `delta`: Option delta

In [10]:
# Example: model_ES with sample data
sample_row = {
    'S': 100.0,      # Spot price
    'X': 100.0,      # Strike
    'v': 0.20,       # Volatility (20%)
    't': 30/365,     # Time to expiry (30 days)
}

result = model_ES(sample_row)
print(f"model_ES result:")
print(f"  Input: S={sample_row['S']}, X={sample_row['X']}, v={sample_row['v']:.0%}, t={sample_row['t']:.4f}")
print(f"  Output: {result}")

model_ES result:
  Input: S=100.0, X=100.0, v=20%, t=0.0822
  Output: {'mv': '-', 'delta': '-'}


---
## 5. Override Expiry

Some assets have non-standard expiry dates that are specified via an overrides CSV file.

The overrides file contains:
- `underlying`: Asset name
- `year`: Expiry year
- `month`: Expiry month
- `date`: Actual expiry date

The `OVERRIDE` expiry code in schedules uses these dates instead of computing from rules.

In [11]:
# Check if overrides file exists and show sample
import os

if os.path.exists(OVERRIDE_PATH):
    import csv
    with open(OVERRIDE_PATH) as f:
        reader = csv.DictReader(f)
        rows = list(reader)[:10]
    
    print(f"Override file: {OVERRIDE_PATH}")
    print(f"Total overrides: {len(rows)}+")
    print("\nSample overrides:")
    for row in rows[:5]:
        print(f"  {row}")
else:
    print(f"Override file not found: {OVERRIDE_PATH}")

Override file: ../data/overrides.csv
Total overrides: 10+

Sample overrides:
  {'ticker': '0R Comdty', 'expiry': '2006-02-10'}
  {'ticker': '0R Comdty', 'expiry': '2006-03-10'}
  {'ticker': '0R Comdty', 'expiry': '2006-04-13'}
  {'ticker': '0R Comdty', 'expiry': '2006-05-12'}
  {'ticker': '0R Comdty', 'expiry': '2006-06-16'}


---
## 6. Cache Management

### `clear_override_cache()`

Clear the expiry override cache. Use when the overrides CSV has been updated.

In [12]:
clear_override_cache()
print("Override cache cleared")

Override cache cleared


---
## 7. Benchmark

Performance of valuation functions.

In [13]:
def run_bench(func, n_runs=5):
    """Run benchmark without data argument."""
    times = timeit.repeat(func, number=1, repeat=n_runs)
    return min(times) * 1000

N_RUNS = 3

In [14]:
# Benchmark: actions()
prices_table = get_prices("CL Comdty", 2024, 6, 0, AMT_PATH, CHAIN_PATH, PRICES_PATH)

# Warmup
_ = actions(prices_table, AMT_PATH, OVERRIDE_PATH)

t_actions = run_bench(
    lambda: actions(prices_table, AMT_PATH, OVERRIDE_PATH),
    N_RUNS
)

n_rows = len(prices_table['rows'])
print(f"actions(): {t_actions:.1f}ms for {n_rows} rows")
print(f"Throughput: {n_rows/t_actions*1000:.0f} rows/sec")

actions(): 0.1ms for 61 rows
Throughput: 865772 rows/sec


In [15]:
# Benchmark: get_straddle_actions()
_ = get_straddle_actions("CL Comdty", 2024, 6, 0, AMT_PATH, CHAIN_PATH, PRICES_PATH, OVERRIDE_PATH)

t_straddle = run_bench(
    lambda: get_straddle_actions("CL Comdty", 2024, 6, 0, AMT_PATH, CHAIN_PATH, PRICES_PATH, OVERRIDE_PATH),
    N_RUNS
)

result = get_straddle_actions("CL Comdty", 2024, 6, 0, AMT_PATH, CHAIN_PATH, PRICES_PATH, OVERRIDE_PATH)
n_rows = len(result['rows'])

print(f"get_straddle_actions(): {t_straddle:.1f}ms for {n_rows} rows")
print(f"Throughput: {n_rows/t_straddle*1000:.0f} rows/sec")

get_straddle_actions(): 0.2ms for 61 rows
Throughput: 252196 rows/sec


In [16]:
# Benchmark: get_straddle_valuation()
_ = get_straddle_valuation("CL Comdty", 2024, 6, 0, AMT_PATH, CHAIN_PATH, PRICES_PATH, OVERRIDE_PATH)

t_val = run_bench(
    lambda: get_straddle_valuation("CL Comdty", 2024, 6, 0, AMT_PATH, CHAIN_PATH, PRICES_PATH, OVERRIDE_PATH),
    N_RUNS
)

result = get_straddle_valuation("CL Comdty", 2024, 6, 0, AMT_PATH, CHAIN_PATH, PRICES_PATH, OVERRIDE_PATH)
n_rows = len(result['rows'])

print(f"get_straddle_valuation(): {t_val:.1f}ms for {n_rows} rows")
print(f"Throughput: {n_rows/t_val*1000:.0f} rows/sec")

get_straddle_valuation(): 0.3ms for 61 rows
Throughput: 197172 rows/sec


In [17]:
# Benchmark: Multiple straddles
assets = ["CL Comdty", "EURUSD Curncy", "SPX Index", "LA Comdty"]

print("Valuation time by asset:")
print("-" * 50)

for asset in assets:
    # Warmup
    _ = get_straddle_valuation(asset, 2024, 6, 0, AMT_PATH, CHAIN_PATH, PRICES_PATH, OVERRIDE_PATH)
    
    t = run_bench(
        lambda a=asset: get_straddle_valuation(a, 2024, 6, 0, AMT_PATH, CHAIN_PATH, PRICES_PATH, OVERRIDE_PATH),
        N_RUNS
    )
    
    result = get_straddle_valuation(asset, 2024, 6, 0, AMT_PATH, CHAIN_PATH, PRICES_PATH, OVERRIDE_PATH)
    n_rows = len(result['rows'])
    
    print(f"{asset:20s}: {t:6.1f}ms ({n_rows} rows)")

Valuation time by asset:
--------------------------------------------------
CL Comdty           :    0.3ms (61 rows)
EURUSD Curncy       :    0.3ms (61 rows)
SPX Index           :    0.2ms (61 rows)
LA Comdty           :    0.3ms (61 rows)


---
## 8. Summary

### Action Functions

| Function | Description |
|----------|-------------|
| `actions(prices_table, path, overrides_csv)` | Add action/strike columns to prices table |
| `get_straddle_actions(underlying, year, month, i, ...)` | Get prices + actions in one call |

### Valuation Functions

| Function | Description |
|----------|-------------|
| `get_straddle_valuation(underlying, year, month, i, ...)` | Get full valuation with PnL columns |

### Pricing Models

| Function | Model | Description |
|----------|-------|-------------|
| `model_ES` | CDS_ES | Expected shortfall for CDS |
| `model_NS` | NS | Normal spread model |
| `model_BS` | BS | Black-Scholes model |
| `MODEL_DISPATCH` | - | Dict mapping model names to functions |

### Cache Functions

| Function | Description |
|----------|-------------|
| `clear_override_cache()` | Clear expiry override cache |

### Output Columns

| Column | Description |
|--------|-------------|
| `action` | Entry/expiry trigger (`ntry`, `xpry`, `-`) |
| `model` | Pricing model code |
| `strike_vol` | Volatility at strike setting |
| `strike` | Strike price |
| `expiry` | Expiry date |
| `mv` | Mark-to-market value |
| `delta` | Option delta |
| `opnl` | Option PnL (change in mv) |
| `hpnl` | Hedge PnL |
| `pnl` | Total PnL (opnl + hpnl) |